In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np


tqdm.pandas()
sns.set_theme()
%matplotlib inline


## Reading in original data

In [2]:
df = pd.read_csv('dataport-export_gas_oct2015-mar2016.csv')

In [3]:
df

,localminute,dataid,meter_value
0,2015-10-01 00:00:10-05,739,88858
1,2015-10-01 00:00:13-05,8890,197164
2,2015-10-01 00:00:20-05,6910,179118
3,2015-10-01 00:00:22-05,3635,151318
4,2015-10-01 00:00:22-05,1507,390354
...,...,...,...
1584818,2016-03-31 23:59:14.336743-05,2129,201726
1584819,2016-03-31 23:59:17.427165-05,2945,161232
1584820,2016-03-31 23:59:35.370782-05,9729,138146
1584821,2016-03-31 23:59:47.816286-05,5129,166488


## Splitting localminute to localdate and localhour

In [ ]:
def split_date_hour(row):
    local_minute = row['localminute']
    row['localdate'], time = local_minute.split(' ')
    _, time = local_minute.split(' ')
    time_list = time.split(':')
    row['localhour'] = time_list[0]
    return row

tqdm.pandas()
df = df.progress_apply(split_date_hour, axis=1)

In [6]:
df

,localminute,dataid,meter_value,localdate,localhour
0,2015-10-01 00:00:10-05,739,88858,2015-10-01,0
1,2015-10-01 00:00:13-05,8890,197164,2015-10-01,0
2,2015-10-01 00:00:20-05,6910,179118,2015-10-01,0
3,2015-10-01 00:00:22-05,3635,151318,2015-10-01,0
4,2015-10-01 00:00:22-05,1507,390354,2015-10-01,0
...,...,...,...,...,...
1584818,2016-03-31 23:59:14.336743-05,2129,201726,2016-03-31,23
1584819,2016-03-31 23:59:17.427165-05,2945,161232,2016-03-31,23
1584820,2016-03-31 23:59:35.370782-05,9729,138146,2016-03-31,23
1584821,2016-03-31 23:59:47.816286-05,5129,166488,2016-03-31,23


In [23]:
# df.drop(columns=['localminute'], inplace=True )
df.to_csv('data_with_date_hour_separate.csv', index=False)

In [24]:
df = pd.read_csv('data_with_date_hour_separate.csv')

## Segmenting data based on dataid

In [47]:
dataid = df.groupby(['dataid', 'localdate', 'localhour']).agg(lambda x : x.tolist())
dataid

meter_value
dataid localdate  localhour                              
35     2015-10-01 0                        [93470, 93470]
                  2          [93470, 93470, 93470, 93470]
                  3                               [93470]
                  4                        [93470, 93470]
                  5                        [93470, 93470]
...                                                   ...
9982   2016-03-28 9                              [119734]
                  11                             [119736]
       2016-03-29 4                              [119784]
       2016-03-30 17                             [119852]
       2016-03-31 11                             [119924]

[397647 rows x 1 columns]

## Getting last detected meter value per hour

In [50]:
def get_diff_per_hour(x):
    x['last_value'] = x['meter_value'][-1] 
    return x

hourly_dataid = dataid.progress_apply(get_diff_per_hour, axis=1)

  0%|          | 0/397647 [00:00<?, ?it/s]

In [51]:
hourly_dataid

meter_value  last_value
dataid localdate  localhour                                          
35     2015-10-01 0                        [93470, 93470]       93470
                  2          [93470, 93470, 93470, 93470]       93470
                  3                               [93470]       93470
                  4                        [93470, 93470]       93470
                  5                        [93470, 93470]       93470
...                                                   ...         ...
9982   2016-03-28 9                              [119734]      119734
                  11                             [119736]      119736
       2016-03-29 4                              [119784]      119784
       2016-03-30 17                             [119852]      119852
       2016-03-31 11                             [119924]      119924

[397647 rows x 2 columns]

In [66]:
hourly_dataid.drop(columns=['meter_value'], inplace=True)

In [67]:
hourly_dataid.to_csv('hourly_data.csv')

In [73]:
daily_data = df.groupby(['dataid', 'localdate']).agg(lambda x : x.tolist()[-1])

In [74]:
daily_data

meter_value  localhour
dataid localdate                         
35     2015-10-01        93484         23
       2015-10-02        93522         23
       2015-10-03        93550         23
       2015-10-04        93588         21
       2015-10-05        93624         23
...                        ...        ...
9982   2016-03-27       119728         23
       2016-03-28       119736         11
       2016-03-29       119784          4
       2016-03-30       119852         17
       2016-03-31       119924         11

[23245 rows x 2 columns]

In [76]:
daily_data.to_csv('daily_data.csv')

In [14]:
hourly_data = pd.read_csv('../hourly_data.csv')
daily_data = pd.read_csv('../daily_data.csv')

In [54]:
hourly_data

,dataid,localdate,localhour,last_value
0,35,2015-10-01,0,93470
1,35,2015-10-01,2,93470
2,35,2015-10-01,3,93470
3,35,2015-10-01,4,93470
4,35,2015-10-01,5,93470
...,...,...,...,...
397642,9982,2016-03-28,9,119734
397643,9982,2016-03-28,11,119736
397644,9982,2016-03-29,4,119784
397645,9982,2016-03-30,17,119852


In [56]:
hourly_data['diff'] = hourly_data.groupby(['dataid'])['last_value'].diff().fillna(0)

In [57]:
hourly_data

,dataid,localdate,localhour,last_value,diff
0,35,2015-10-01,0,93470,0.0
1,35,2015-10-01,2,93470,0.0
2,35,2015-10-01,3,93470,0.0
3,35,2015-10-01,4,93470,0.0
4,35,2015-10-01,5,93470,0.0
...,...,...,...,...,...
397642,9982,2016-03-28,9,119734,0.0
397643,9982,2016-03-28,11,119736,2.0
397644,9982,2016-03-29,4,119784,48.0
397645,9982,2016-03-30,17,119852,68.0


In [58]:
hourly_data.to_csv('hourly_data2.csv', index=False)

dataid          int64
localdate      object
meter_value     int64
localhour       int64
dtype: object

In [50]:
daily_data['diff'] = daily_data.groupby(['dataid'])['meter_value'].diff().fillna(0)

In [53]:
daily_data.to_csv('daily_data2.csv', index=False)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
23240   NaN
23241   NaN
23242   NaN
23243   NaN
23244   NaN
Name: meter_value, Length: 23245, dtype: float64